In [1]:
from keras.callbacks import ModelCheckpoint
from RecommendationModel import RecommendationModel
from DataPreprocessing import DataPreprocessing

Using TensorFlow backend.


In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!unzip "/content/gdrive/My Drive/CollaborativeFiltering/movielens-20m-dataset.zip"

Archive:  /content/gdrive/My Drive/CollaborativeFiltering/movielens-20m-dataset.zip
  inflating: movie.csv               
  inflating: link.csv                
  inflating: tag.csv                 
  inflating: genome_tags.csv         
  inflating: rating.csv              
  inflating: genome_scores.csv       


In [0]:
dataPreprocessing = DataPreprocessing()

#getting training data
userIds, movieIds, ratings, rating_csv, movie_csv = dataPreprocessing.load_data()

In [6]:
#getting emmbeding model with Dense layers
rModel = RecommendationModel(dataPreprocessing.max_userId, dataPreprocessing.max_movieId, dataPreprocessing.k_factor)
model = rModel.generate_embeddedModel()
model.compile(loss='mse',
              optimizer='adamax')
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
embedding_1_input (InputLayer)  (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_2_input (InputLayer)  (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 128)       17727104    embedding_1_input[0][0]          
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 128)       16801536    embedding_2_input[0][0]          
_____________________________________

In [7]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.0 GB  | Proc size: 4.1 GB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


In [10]:
############## Training part ################
print('userIds : ',userIds, ' shape : ',userIds.shape)
print('movieIds : ',movieIds, ' shape : ',movieIds.shape)
print('ratings : ',ratings, ' shape : ',ratings.shape)
print('start training...........')
filepath = 'weights_best_embedded.hdf5'
checkPoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
callbacks_lst = [checkPoint]
model.fit([userIds, movieIds], ratings, validation_split=0.1, epochs=10, batch_size=512, callbacks=callbacks_lst)

userIds :  [ 41196 121765 122597 ... 117582 105673 114726]  shape :  (20000263,)
movieIds :  [ 1952 58559  1219 ...  6375   597   527]  shape :  (20000263,)
ratings :  [5. 4. 5. ... 5. 3. 3.]  shape :  (20000263,)
start training...........
Train on 18000236 samples, validate on 2000027 samples
Epoch 1/10
18000236/18000236 [==============================] - 742s 41us/step - loss: 3.8209 - val_loss: 1.0774

Epoch 00001: val_loss improved from inf to 1.07744, saving model to weights_best_embedded.hdf5
Epoch 2/10
18000236/18000236 [==============================] - 739s 41us/step - loss: 0.9130 - val_loss: 0.8350

Epoch 00002: val_loss improved from 1.07744 to 0.83504, saving model to weights_best_embedded.hdf5
Epoch 3/10
18000236/18000236 [==============================] - 738s 41us/step - loss: 0.7827 - val_loss: 0.7623

Epoch 00003: val_loss improved from 0.83504 to 0.76230, saving model to weights_best_embedded.hdf5
Epoch 4/10
18000236/18000236 [==============================] - 738s 4

In [0]:
model_save_name = ''
path = F"/content/gdrive/My Drive/CollaborativeFiltering/{filepath}" 
model.save_weights(path)